In [ ]:
!wget "https://github.com/sasaa09/news_tweet_classification/raw/master/model.pkl"

In [ ]:
!pip install flask-ngrok
!pip install -U flask-cors

### Fetch Recent Tweets

In [ ]:
consumer_key = 'HesPOfbPcUp3mc6D9aRvItAnU'
consumer_secret = 'nQ0owQEVXLwgJEwfO50ifrEYD9zyXDD37Jf3SIwF9ugKkbnFV9'
access_token = '307762558-VXeWvKTZEsIaVYX0yCDzVm8gw1aJOyPp2hC4PGgw'
access_token_secret = 'jFgyvRlWu4FjcDBXhIC7ngZjq7ri1lrAdvjlngZp6bjJB'

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
list_of_news_accounts = [
                          'cnn',
                          'nytimes',
                          'bbcbreaking',
                          'bbcworld',
                          'theeconomist',
                          'telegraph',
                          'guardiannews',
                          'cnbc',
                          'ajenglish',
                          'breakingnews',
                          'huffpost',
                          'washingtonpost',
                          'abc',
                          'wsj',
                          'time'
                        ]

In [ ]:
import pickle

model = pickle.load(open('model.pkl','rb'))

def predict_category(tweet):
  tweet_text = tweet['text']
  tweet_category = model.predict([tweet_text])[0]
  tweet_classified = tweet
  tweet_classified['category'] = tweet_category
  return tweet_classified

In [ ]:
tweets_raw = []
for news_account in list_of_news_accounts:
  tweets_in_news_account = api.user_timeline(id=news_account)
  for tweet in tweets_in_news_account:
    tweets_raw.append(tweet)

import re

tweets = []
for tweet_raw in tweets_raw:
  tweet_text = re.sub(r'http\S+', '', tweet_raw.text)
  tweet_text = re.sub(r'[^A-Za-z0-9 ]+', '', tweet_text)
  tweet_text = tweet_text.replace('  ', ' ')
  tweet = {
      'url': 'https://twitter.com/twitter/statuses/' + tweet_raw.id_str,
      'username': tweet_raw.user.name.lower(),
      'text': tweet_text,
      'time': tweet_raw.created_at,
      'likes': tweet_raw.favorite_count,
      'retweets': tweet_raw.retweet_count,
      'profile_picture': tweet_raw.user.profile_image_url_https
  }
  tweet_classified = predict_category(tweet)
  tweets.append(tweet_classified)

### Endpoints

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, request
from flask_cors import CORS
import codecs
import csv
import pandas as pd

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

@app.route("/")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"

@app.route('/tweets', methods=['GET'])
def get_all_tweets():
    return { 'tweets': tweets }
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5f62fe9eaa62.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Jul/2021 13:51:13] "GET /tweets HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2021 14:30:53] "GET /tweets HTTP/1.1" 200 -
